### Pregunta 2
En esta pregunta creé 2 clases para implementar RSA basándome en los algoritmos de secuencia de encriptación y desencriptación y el test de primalidad que se vieron en clases. __RSASender__ se encarga de encriptar un mensaje según una llave pública que recibe y __RSAReceiver__ genera esta llave y genera una secreta para poder desencriptar el mensaje que le mandaron. Además de estas clases, para generar los números primos grandes _P_ y _Q_, además de _N_, _Phi_, _d_ y _e_ implementé las funciones que están a continuación. Finalmente, para obtener el inverso de _d_(_e_) usé la función de inverso que usa el algoritmo extendido de Euclides la cual estaba en el repositorio del año pasado.

In [ ]:
import random
import math

In [ ]:
def _generar_primo(bits_largo):
    while True:
        primo = random.randint(2**(bits_largo - 1), 2**bits_largo - 1) # número random con bits_largo de largo
        if _test_primalidad(primo, 100): 
            return primo

In [ ]:
def _test_primalidad(num, k):
    # se quitan los casos triviales, que no sea 1, par o potencia de un número. 2 devuelve True inmediatamente.
    if num == 1:
        return False
    elif num == 2:
        return True
    elif num % 2 == 0:
        return False
    elif _es_potencia(num):
        return False
    else:
        neg = 0
        for i in range(k):
            pos_div = random.randint(2, num-1)
            if math.gcd(pos_div, num) > 1:
                return False
            else:
                b = pow(pos_div, (num-1)//2, num)
                if b == num - 1:
                    neg += 1
                elif b != 1:
                    return False
        if neg > 0:
            return True
        else:
            return False        

In [ ]:
def _es_potencia(n): # función que hace búsqueda binaria desde el 2 hasta el mayor exponente posible buscando una posible base.
    limite = math.log(n, 2)
    exp = 2
    lo = 0
    hi = n
    mid = 0
    while exp <= limite:
        while lo <= hi:
            mid = (hi + lo)// 2
            if mid**exp < n:
                lo = mid + 1
            elif mid**exp > n:
                hi = mid -1 
            else:
                return True
        exp += 1
    return False

In [ ]:
def _generar_clave(largo):
    P = _generar_primo(largo // 2 + 1) # se generan 2 primos P y Q de largo/2 + 1 bits
    Q = _generar_primo(largo // 2 + 1)
    N = P * Q
    Phi = (P - 1) * (Q - 1)
    d = random.randint(1, N - 1)

    while (math.gcd(d, Phi) > 1):
        d = random.randint(1, N - 1)

    e = _inverso(d, Phi)
    return N, d, e

In [ ]:
def _inverso(a: int, n: int) -> int: 
    (r, s, t) = _alg_ext_euclides(a, n)
    return s % n

In [ ]:
def _alg_ext_euclides(a: int, b: int) -> (int, int, int):
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1 > 0:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

In [ ]:
class RSAReceiver :
    def __init__(self, bit_len: int):
        """
        Arguments :
        bit_len : A lower bound for the number of bits of N,
        the second argument of the public and secret key .
        """
        self.largo = bit_len 
        self.N, self.d, self.e = _generar_clave(self.largo) # se genera la clave al inicializar la instancia
        self.len_N = int(math.log(self.N,2)//8 +1) # largo de N

    def get_public_key(self) -> bytearray:        
        """
        Returns :
        public_key : Public key expressed as a Python ’bytearray ’ using the
        PEM format . This means the public key is divided in:
        (1) The number of bytes of e (4 bytes )
        (2) the number e (as many bytes as indicated in (1))
        (3) The number of bytes of N (4 bytes )
        (4) the number N (as many bytes as indicated in (3))
        """
        # se calculan los largos y se juntan con los números en un bytearray
        bytes_e = self._bytes_of_number(self.e)
        bytes_N = self._bytes_of_number(self.N)
        e_bytes = self._number_to_bytes(self.e)
        N_bytes = self._number_to_bytes(self.N)
        return bytes_e + e_bytes + bytes_N + N_bytes
    
    def decrypt(self, ciphertext: bytearray) -> str:
        """
        Arguments :
        ciphertext : The ciphertext to decrypt
        Returns :
        message : The original message
        """
        mensaje = ""
        cont = 0
        # se hace el proceso inverso de encriptar, se vuelve a bytes un bloque de tamaño n + 1,
        # se calcula c^d para obtener el número original del mensaje, se vuelve a bytes ese número
        # y finalmente se transforma en string.
        while cont < len(ciphertext):
            num_c = int.from_bytes(ciphertext[cont:cont + self.len_N], "big")
            dec_num = pow(num_c, self.d, self.N)
            array = dec_num.to_bytes(self.len_N-1,"big")
            if cont + self.len_N >= len(ciphertext):
                for i in range(len(array)):
                    if array[i] != 0:
                        array = array[i:]
                        break
            mensaje += array.decode("utf-8")
            cont += self.len_N
            
        return mensaje
        
    def _bytes_of_number(self, num): # número de bytes que ocupa un número
        return bytearray(int(math.log(num,2)//8 +1).to_bytes(4,"big"))
    
    def _number_to_bytes(self, num): # transformación de número a bytes
        return bytearray(num.to_bytes(int(math.log(num,2)//8 +1),"big"))

In [ ]:
class RSASender:
    def __init__(self, public_key: bytearray):
        """
        Arguments :
        public_key : The public key that will be used to encrypt messages
        """
        self.public_key = public_key
        self.N = 0
        self.e = 0
    
    def encrypt(self, message: str) -> bytearray :
        """
        Arguments :
        message : The plaintext message to encrypt
        Returns :
        ciphertext : The encrypted message
        """
        # se separa la llave en las partes necesarias para encriptar el mensaje.
        len_e = int.from_bytes(self.public_key[:4], "big")
        self.e = int.from_bytes(self.public_key[4:4+len_e], "big")
        len_N = int.from_bytes(self.public_key[4+len_e:4+len_e+4], "big")
        self.N = int.from_bytes(self.public_key[4+len_e+4:4+len_e+4+len_N], "big")
        
        l_block = len_N - 1 # cada bloque debe tener máximo 1 byte menos que los que necesita N
        cont = 0
        # se encripta pasando el string a bytes, luego se separa en bloques y cada uno se transforma en un número que se eleva
        # a e (m^e) para conseguir c, ese número se transforma de nuevo a bytes en un bloque de 1 más de largo y finalmente
        # se junta todo para retornarlo.
        mensaje = bytearray(message, "utf-8")
        encriptado = bytearray()
        while cont < len(mensaje):
            bloque = mensaje[cont:cont+l_block]
            num_a = int.from_bytes(bloque, "big")
            num_c = pow(num_a, self.e, self.N)
            encriptado += num_c.to_bytes(len_N, "big")
            cont += l_block
        
        return encriptado